In [1]:
import pandas as pd
import difflib

In [2]:
## DATA FRAME DE TODO
df1 = pd.read_csv('CABA-CPAS.txt')
df2 = pd.read_csv('BSAS-CPAS.txt')
df = pd.concat([df1, df2])
df['CP'] = pd.to_numeric(df['CP'])

df['CALLE'] = df['CALLE'].str.lower()
df['PAR'] = df['PAR']=='números pares'
df.head(1)

,CALLE,DESDE,HASTA,PAR,CP,CPA,PROVINCIA,LOCALIDAD
0,calle juana de ibarbourou,1001,1099,False,1405,C1405BZA,capital-federal,ciudad-autonoma-buenos-aires


In [3]:
## DATA FRAME MUESTRA USERS
df_test = pd.read_csv('lemoncash_ar_Accounts.csv')
df_test['CALLE'] = df_test['CALLE'].str.lower()
df_test['PAR'] = (df_test['NUM'] % 2) == 0
df_test = pd.concat([df_test[df_test['PROV'] == 'Buenos Aires'],df_test[df_test['PROV'] == 'Provincia de Buenos Aires']])
df_test.head(1)

,CALLE,NUM,LOCALIDAD,PROV,CP,PAR
0,olleros,1800.0,Palermo,Buenos Aires,1426,True


In [4]:
df_test.count()[0]

438

In [5]:
# Funcion para buscar la mejor coincidencia de nombre de calle
def check(x):
    df_new = df.loc[df['CP'] == x['CP']]
    bestmatch = difflib.get_close_matches(x['CALLE'], df_new['CALLE'], 1, 0.3)
    return bestmatch

In [6]:
## Busco la calle más similar en mi dataset 
df_test['bestmatch'] = df_test.apply(lambda x: check(x), axis=1)
df_test.head()

,CALLE,NUM,LOCALIDAD,PROV,CP,PAR,bestmatch
0,olleros,1800.0,Palermo,Buenos Aires,1426,True,[calle olleros]
25,calle muzilli,1235.0,Pilar Centro,Buenos Aires,1635,False,[calle italia]
27,maipú,812.0,Comuna 1,Buenos Aires,1057,True,[]
28,corrientes,NaN,Buenos Aires,Buenos Aires,3365,False,[]
34,bolivia,3750.0,Comuna 15,Buenos Aires,1419,True,[calle bolivia]


In [7]:
# Me quedo solo con los que encontré una calle similar
df_test = df_test[df_test['bestmatch'].str.len()>0]
df_test.count()[0]

371

In [8]:
# Funcion para buscar el CPA
def find(x):
    df_new = df[df['CALLE'] == x['bestmatch'][0]]
    df_new = df_new[df_new['CP'] == x['CP']]
    df_new = df_new[(df_new["DESDE"]<=x['NUM']) & (df_new["HASTA"]>=x['NUM'])]
    df_new = df_new[df_new['PAR']==x['PAR']]
    return df_new['CPA'].values


In [9]:
## Busco el CPA
df_test['CPA Obtenido'] = df_test.apply(lambda x: find(x), axis=1)

In [10]:
## Los que tienen CPA
df_test = df_test[df_test['CPA Obtenido'].str.len()>0]
df_test.head()

,CALLE,NUM,LOCALIDAD,PROV,CP,PAR,bestmatch,CPA Obtenido
0,olleros,1800.0,Palermo,Buenos Aires,1426,True,[calle olleros],[C1426CRF]
25,calle muzilli,1235.0,Pilar Centro,Buenos Aires,1635,False,[calle italia],[B1635ILC]
34,bolivia,3750.0,Comuna 15,Buenos Aires,1419,True,[calle bolivia],[C1419DTF]
35,maipú,407.0,Comuna 1,Buenos Aires,1006,False,[calle maipu],[C1006ACC]
53,enrique ochoa,1052.0,Comuna 4,Buenos Aires,1437,True,[calle enrique ochoa],[C1437DUR]


In [11]:
df_test.to_csv('cpas.csv', sep='\t')